In [1]:
import sys
from copy import deepcopy
import time

sys.path.append('../utils')
from pyutils import *

In [2]:
sample = readutf8('sample.txt')
sample_long = readutf8('sample2.txt')
puzzle = readutf8('input.txt')

In [3]:
def parse_input(puzzle_in: str) -> tuple[StrMatrix, list[Point], Point]:
    warehouse_map: StrMatrix = []
    robot_instructions: list[Point] = []
    robot_pos: Point = (-1, -1)
    char_to_point = {'^': (-1, 0), '>': (0, 1), 'v': (1, 0), '<': (0, -1)}
    part: int = 1
    for line in puzzle_in.split('\n'):
        if line.strip() == '':
            part += 1
            continue
        if part == 1:
            wide_line = []
            for ch in line:
                match ch:
                    case '#': wide_line.extend('##')
                    case 'O': wide_line.extend('[]')
                    case '.': wide_line.extend('..')
                    case '@': wide_line.extend('@.')
            warehouse_map.append(wide_line)
            if '@' in wide_line:
                robot_pos = (len(warehouse_map) - 1, wide_line.index('@'))
        elif part == 2:
            robot_instructions.extend(list(map(lambda i: char_to_point[i], line)))
    if robot_pos == (-1, -1):
        raise ValueError('Robot position not found in warehouse map!')
    return (warehouse_map, robot_instructions, robot_pos)

In [30]:
def predict_robot(mat: StrMatrix, instructions: list[Point], robot_start: Pt):
    mat = deepcopy(mat)
    def _check_boxes(box_pair: tuple[Pt, Pt]) -> bool:
        print('CHECKBOXCALL')
        box_future = (point_op(add, box_pair[0], dirpt), point_op(add, box_pair[1], dirpt))
        print(box_pair)
        print(box_future)
        time.sleep(1)
        ret = True
        # future of [
        if box_future[0] == box_pair[1]:
            ret = True
        elif box_future[0] in checked:
            ret = checked[box_future[0]]
        else:
            match (obj_a := matget(mat, box_future[0])):
                case '#': ret = False
                case '.': ret = True
                case '[' | ']':
                    other_pair = (box_future[0], box_future[1]) if obj_a == '[' else (box_future[0] + (0, -1), box_future[0])
                    ret = _check_boxes(other_pair)
                case _: raise ValueError(f'Unexpected character at {box_future!r}: {obj_a!r}')
            checked[box_future[0]] = ret
        if ret == False:
            return ret
        # future of ]
        if box_future[1] == box_pair[0]:
            ret = True
        elif box_future[1] in checked:
            ret = checked[box_future[1]]
        else:
            match (obj_b := matget(mat, box_future[1])):
                case '#': ret = False
                case '.': ret = True
                case '[' | ']':
                    other_pair = (box_future[1], box_future[1] + (0, 1)) if obj_b == '[' else (box_future[0], box_future[1])
                    ret = _check_boxes(other_pair)
                case _: raise ValueError(f'Unexpected character at {box_future!r}: {obj_b!r}')
            checked[box_future[1]] = ret
        if ret == True:
            matset(mat, box_pair[0], '.')
            matset(mat, box_pair[1], '.')
            matset(mat, box_future[0], '[')
            matset(mat, box_future[1], ']')
        return ret

    robot_pos = robot_start
    # for dirpt in instructions:
    for n, dirpt in enumerate(instructions):
        time.sleep(2)
        clear_output(wait=True)
        print(dirpt)
        print(f'{n:<8} / {len(instructions) - 1:<8}')
        print(mat_restring(mat))
        checked: dict[Point, bool] = {}
        robot_future = point_op(add, robot_pos, dirpt)
        match (obj := matget(mat, robot_future)):
            case '#': continue
            case '.':
                matset(mat, robot_pos, '.')
                robot_pos = robot_future
                matset(mat, robot_pos, '@')
            case '[' | ']':
                box_pair = (robot_future, robot_future + (0, 1)) if obj == '[' else (robot_future + (0, -1), robot_future)
                print(f'.....{box_pair}')
                result = _check_boxes(box_pair)
                if result == True:
                    matset(mat, robot_pos, '.')
                    robot_pos = robot_future
                    matset(mat, robot_pos, '@')

    return mat

In [31]:
def box_gps_coords(mat: StrMatrix) -> list[int]:
    coords: list[int] = []
    for pt, i in mat_iter(mat):
        if i == 'O':
            coords.append((100 * pt[0]) + pt[1])
    return coords

In [32]:
warehouse, robot_moves, robot = parse_input(sample_long)
print(mat_restring(warehouse))

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################


In [33]:
x = predict_robot(warehouse, robot_moves, robot)

(0, -1)
0        / 699     
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
.....((4, 7, 0, -1), (4, 7))
CHECKBOXCALL
((4, 7, 0, -1), (4, 7))
((4, 6), (4, 6))
CHECKBOXCALL
((4, 6), (4, 6))
((4, 5), (4, 5))


KeyboardInterrupt: 

In [24]:
# print(sum(box_gps_coords(predict_robot(warehouse, robot_moves, robot))))